In [2]:
import numpy as np

epsilon = 0.001
epochs = 1


def load_data(filename):
    X_data = []
    y_data = []
    with open(filename) as file:
        for line in file:
            splittedLine = line.split(" ")
            X = splittedLine[:256]
            X = np.array([0 if float(i) == 0. else 1 for i in X])
            X_data.append(X)

            y = splittedLine[256:-1]
            y = [int(i) for i in y]
            y = y.index(1)
            y_data.append(y)

    return np.array(X_data), np.array(y_data)


def shuffle(a, b):
    pos = np.random.permutation(len(a))
    return a[pos], b[pos]


def addComplement(X):
    X_data = []
    for x in X:
        complement = np.array([i ^ 1 for i in x])
        X_data.append(np.concatenate((x, complement), axis=None))

    return np.array(X_data)


def calculate_T(I, w, alpha):
    T = np.sum(np.minimum(I, w)) / (alpha + np.sum(w))
    return T


def addNeuron(layer, n):
    w = np.ones(n)
    layer.append([w, 0, -1])  # weights, is commited, class
    return layer


def run(X, y, alpha, beta, vigilance):
    second_layer = []

    for epoch in range(epochs):
        for it in range(X.shape[0]):
            if len(second_layer) == 0:
                second_layer = addNeuron(second_layer, X.shape[1])

            T = []
            step = 1
            J = -1
            l_vigilance = -1
            while True:
                if step == 1:
                    l_vigilance = vigilance
                    step = 2
                if step == 2:
                    for i in range(len(second_layer)):
                        if second_layer[i][1] == 0:
                            t = np.random.random()
                        else:
                            t = calculate_T(X[it], second_layer[i][0], alpha)
                        T.append(t)

                    step = 3
                if step == 3:
                    J = np.argmax(T)

                    if second_layer[J][1] == 0:
                        step = 7
                    else:
                        step = 4
                if step == 4:
                    if np.sum(np.minimum(X[it], second_layer[J][0])) / np.sum(X[it]) >= l_vigilance:
                        step = 5
                    else:
                        T[J] = -1
                        step = 3
                if step == 5:
                    if second_layer[J][2] == y[it]:
                        w_new = beta * np.minimum(X[it], second_layer[J][0]) + (1 - beta) * second_layer[J][0]
                        second_layer[J][0] = w_new
                        step = 9
                    else:
                        T[J] = -1
                        l_vigilance = np.sum(np.minimum(X[it], second_layer[J][0])) / np.sum(X[it]) + epsilon
                        step = 6
                if step == 6:
                    if l_vigilance > 1:
                        step = 9
                    else:
                        step = 3
                if step == 7:
                    second_layer[J][0] = X[it]
                    second_layer[J][1] = 1
                    second_layer[J][2] = y[it]
                    step = 8
                if step == 8:
                    second_layer = addNeuron(second_layer, X.shape[1])
                    step = 9
                if step == 9:
                    break

    return second_layer


def test(second_layer, X, y, alpha):
    nr = 0
    for it in range(X.shape[0]):
        T = []
        for i in range(len(second_layer)):
            t = calculate_T(X[it], second_layer[i][0], alpha)
            T.append(t)

        J = np.argmax(T)
        if second_layer[J][2] == y[it]:
            nr += 1

    return 100 * nr / X.shape[0]


def main():
    X, y = load_data("semeion.data");
    X, y = shuffle(X, y)
    X = addComplement(X)

    nr_train = int(70 * X.shape[0] / 100)
    X_train = X[:nr_train, :]
    y_train = y[:nr_train]
    X_test = X[nr_train:, :]
    y_test = y[nr_train:]

    nr_train_train = int(70 * X_train.shape[0] / 100)
    X_train_train = X_train[:nr_train_train, :]
    X_train_valid = X_train[nr_train_train:, :]
    y_train_train = y_train[:nr_train_train]
    y_train_valid = y_train[nr_train_train:]

    alpha = [0.01, 0.1, 0.2, 0.3, 0.4]
    beta = [1, 0.8, 0.6, 0.4]
    vigilance = [0.01, 0.1, 0.2, 0.3]

    best_alpha = -1
    best_beta = -1
    best_vigilance = -1
    maximum_accuracy = -1

    for al in alpha:
        for be in beta:
            for vi in vigilance:
                second_layer = run(X_train_train, y_train_train, al, be, vi)
                acc = test(second_layer[:-1], X_train_valid, y_train_valid, al)
                print(f"Param: alpha= {al}, beta= {be}, vigilance= {vi}")
                print(f"Accuracy train validation= {acc}")
                if acc > maximum_accuracy:
                    best_alpha = al
                    best_beta = be
                    best_vigilance = vi
                    maximum_accuracy = acc

    second_layer = run(X_train, y_train, best_alpha, best_beta, best_vigilance)
    acc = test(second_layer[:-1], X_test, y_test, best_alpha)
    print("------------------------------------------------------------------------------")
    print(f"Param: alpha= {best_alpha}, beta= {best_beta}, vigilance= {best_vigilance}")
    print(f"Acuracy test= {acc}")


if __name__ == '__main__':
    main()


Param: alpha= 0.01, beta= 1, vigilance= 0.01
Accuracy train validation= 75.5223880597015
Param: alpha= 0.01, beta= 1, vigilance= 0.1
Accuracy train validation= 77.61194029850746
Param: alpha= 0.01, beta= 1, vigilance= 0.2
Accuracy train validation= 83.58208955223881
Param: alpha= 0.01, beta= 1, vigilance= 0.3
Accuracy train validation= 89.25373134328358
Param: alpha= 0.01, beta= 0.8, vigilance= 0.01
Accuracy train validation= 74.02985074626865
Param: alpha= 0.01, beta= 0.8, vigilance= 0.1
Accuracy train validation= 78.2089552238806
Param: alpha= 0.01, beta= 0.8, vigilance= 0.2
Accuracy train validation= 84.4776119402985
Param: alpha= 0.01, beta= 0.8, vigilance= 0.3
Accuracy train validation= 90.74626865671642
Param: alpha= 0.01, beta= 0.6, vigilance= 0.01
Accuracy train validation= 69.85074626865672
Param: alpha= 0.01, beta= 0.6, vigilance= 0.1
Accuracy train validation= 76.41791044776119
Param: alpha= 0.01, beta= 0.6, vigilance= 0.2
Accuracy train validation= 84.17910447761194
Param: 

KeyboardInterrupt: 